In [1]:
import pandas as pd

In [120]:
def decode_openmetrics(input: str) -> dict:
    tokenized = input.split()
    source = tokenized[0]
    value = float(tokenized[1])
    if len(tokenized)>2:
        timestamp = float(tokenized[2])
    else:
        timestamp = None
    source_tokenized = source.split("{")
    metricname = source_tokenized[0]
    if len(source_tokenized)>1:
        tags = source_tokenized[1].split("}")[0].split(",")
    else:
        tags = []
    return {'source':source,'metricname': metricname,'tags': tags, 'timestamp':timestamp, 'value':value}

In [126]:
class InterArrivalTransformer:
    def __init__(self):
        self.latest_arrivaltime_dict = dict()

    # NOTE: this is implemented for streaming, so it transform one event at a time,
    # i.e, the function input is a pandas dataframe with a single row. If multiple rows are provided
    # they are iterated with iterrows(). It should be optimized if the function transform full dataframes. 
    def transform(self, input: pd.DataFrame) -> pd.DataFrame:
        interarrival_rows = []
        for index,row in input.iterrows():
          
            source = row['source']
            timestamp = row['timestamp']
            latest_timestamp = self.latest_arrivaltime_dict.get(source, None)
            self.latest_arrivaltime_dict[source] = timestamp
            if latest_timestamp:
                interarrival_rows.append([source, timestamp, (timestamp - latest_timestamp)])   
        return pd.DataFrame(interarrival_rows, 
                 columns= ['source','timestamp','value'])

## Loading anomaly detection model


In [122]:
#We want a model with incremental learning, and mlfow does not seem to serialize the model with the fit function!
#We disable for now loading mlflow model from model_path. 

import mlflow.pyfunc
import pandas as pd
import numpy as np

# Load the model in `python_function` format
minmax_model = mlflow.pyfunc.load_model('unfitted_minmax_model')

/home/jovyan/.local/lib/python3.7/site-packages/mlflow/pyfunc/__init__.py:281: DeprecationWarning: .. Warning:: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a near future release. Use ``mlflow.pyfunc.load_model`` instead.
  return load_pyfunc(model_uri, suppress_warnings)


In [196]:
import mlflow
import pandas as pd
import numpy as np

class MultisourceMinMaxDetector(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = {}
    
    def fit(self, input: pd.DataFrame):
        #NOTE implemented to allow calling fit multiple times and update model
        min_max_df = input.groupby(['source']).agg({'value':[np.min,np.max]})
        for source,row in min_max_df.iterrows():
            min = row[('value','amin')]
            max = row[('value','amax')]
            if source not in self.model:
                margin =0.05*min
                self.model[source] = {'min': min-margin,
                                      'max': max+margin}
            else:
                margin = 0.05*self.model[source]['min']
                if max > self.model[source]['max']:
                    self.model[source]['max'] = max + margin 
                if min < self.model[source]['min']:
                    self.model[source]['min'] = min - margin
   
    def reset(self):
        self.model={}
    
    def reset_fit(self, input: pd.DataFrame):
        min_max_df = input.groupby(['source']).agg({'value':[np.min,np.max]})
        self.model = {k:v for k,v in zip(min_max_df.index.tolist(),zip(min_max_df[('value', 'amin')].tolist(),min_max_df[('value', 'amax')].tolist()))}

    def predict(self, context, model_input: pd.DataFrame) -> pd.DataFrame:
        #NOTE can be optimized using pandas to perform this condition check
        output = []
        for index,row in model_input.iterrows():  
            source = row['source']
            value = row['value']
            timestamp = row['timestamp']
            if source not in self.model:
                output_row = [source, str(timestamp), value, True, 'unknown source']
            else:
                min = self.model[source]['min']
                max = self.model[source]['max']
                if  min <= value <= max:
                    output_row = [source, str(timestamp), value, False, None]
                else:
                    output_row = [source, str(timestamp), value, True, 'value out of limits ({:.2f},{:.2f})'.format(min,max)]
            output.append(output_row)
        return pd.DataFrame(output, 
                 columns= ['source','timestamp','value','anomaly','anomaly_type'])

minmax_model=MultisourceMinMaxDetector()

# Model Fitting

In [197]:
#reset previous limits
minmax_model.reset()

In [200]:
from kafka import KafkaConsumer
brokers='pnda-cp-kafka:9092'

interarrival_transformer = InterArrivalTransformer()
consumer = KafkaConsumer('openmetrics', bootstrap_servers=brokers.split())
# Fitting Model
try:
   for msg in consumer:
        openmetric_sample=decode_openmetrics(msg.value.decode('utf-8'))
        if openmetric_sample['timestamp'] == None:
            openmetric_sample['timestamp'] = msg.timestamp
        openmetric_df = pd.DataFrame.from_dict(openmetric_sample)
        interrarival_df = interarrival_transformer.transform(openmetric_df)
        minmax_model.fit(interrarival_df)
except KeyboardInterrupt:
    print("Model Fitting Stage interrupted.")
    print("model:       min         max  source")
    for k,v in minmax_model.model.items():
        print("      {:10.2f}  {:10.2f}  {}".format(v['min'],v['max'],k))
    

Model Fitting Stage interrupted.
model:       min         max  source
          956.65     1057.35  temperature{room=corridor}
         1904.75     2105.25  temperature{room=bathroom}
         2187.85     2418.15  humidity{room=kitchen}
         4757.60     5258.40  humidity{room=bathroom}
         3046.65     3367.35  temperature{room=kitchen}
         3806.65     4207.35  humidity{room=corridor}
         1903.80     2104.20  temperature{room=bedroom}


In [ ]:
#Getting anomalies from source

from kafka import KafkaConsumer
brokers='pnda-cp-kafka:9092'

interarrival_transformer = InterArrivalTransformer()

consumer = KafkaConsumer('openmetrics', bootstrap_servers=brokers.split())

try:
    for msg in consumer:
        openmetric_sample=decode_openmetrics(msg.value.decode('utf-8'))
        if openmetric_sample['timestamp'] == None:
            openmetric_sample['timestamp'] = msg.timestamp
        openmetric_df = pd.DataFrame.from_dict(openmetric_sample)
        anomalies_df = minmax_model.predict(context=[],model_input=interarrival_transformer.transform(openmetric_df))
        if len(anomalies_df[anomalies_df.anomaly == True]) > 0:
            print(anomalies_df[anomalies_df.anomaly == True])

except KeyboardInterrupt:
    print("Anomaly Detection stopped.")


    

                      source      timestamp  value  anomaly  \
0  temperature{room=bedroom}  1570714557133   1295     True   

                            anomaly_type  
0  value out of limits (1903.80,2104.20)  
                      source      timestamp  value  anomaly  \
0  temperature{room=bedroom}  1570714557841    708     True   

                            anomaly_type  
0  value out of limits (1903.80,2104.20)  
                      source      timestamp  value  anomaly  \
0  temperature{room=bedroom}  1570714559136   1295     True   

                            anomaly_type  
0  value out of limits (1903.80,2104.20)  
                      source      timestamp  value  anomaly  \
0  temperature{room=bedroom}  1570714559845    709     True   

                            anomaly_type  
0  value out of limits (1903.80,2104.20)  
                      source      timestamp  value  anomaly  \
0  temperature{room=bedroom}  1570714561142   1297     True   

                     